# 0. Imports

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.gridspec as gridspec

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from kneed import KneeLocator
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from collections import Counter
from tabulate import tabulate
from tsfeatures import tsfeatures
from scipy.stats import uniform
from mango import Tuner
from mango import scheduler
from contextlib import contextmanager
import sys

In [25]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [26]:
# Configure seaborn plot style: set background color and use dark grid
sns.set(rc={'axes.facecolor':'#E6E6E6'}, style='darkgrid')

In [27]:
df_train = pd.read_csv("data/train_clustered.csv", index_col=0)
df_test = pd.read_csv("data/test_clustered.csv", index_col=0)
cluster = pd.read_csv("data/clustered_products.csv", index_col=0)

In [28]:
df_train.ds = pd.to_datetime(df_train.ds, format="%Y-%m-%d")
df_test.ds = pd.to_datetime(df_test.ds, format="%Y-%m-%d")

In [29]:
nb_clusters = df_train.cluster.nunique()

# IV. Model selection per cluster

In [30]:
# Check if timing file exists
if os.path.exists("timing.csv"):
    timing = pd.read_csv("timing.csv", index_col=0)
else:
    timing = pd.DataFrame(columns=['time_started', 'type of run', 'time_ended', 'time_elapsed'])

In [31]:
from mango import Tuner
import parameters
import importlib

importlib.reload(parameters)
from parameters import param_spaces, optimizer_configs

def evaluate_models_on_centroid(train, test):
    results = {}

    def objective_function(args_list):
        results = []
        for params in args_list:
            try:
                if model == "SARIMA":
                    order = (params['p'], params['d'], params['q'])
                    seasonal_order = (params['P'], params['D'], params['Q'], params['s'])
                    model_instance = SARIMAX(train, order=order, seasonal_order=seasonal_order).fit(disp=False)
                elif model == "ExponentialSmoothing":
                    model_instance = ExponentialSmoothing(
                        train,
                        seasonal=params['seasonal'],
                        seasonal_periods=params['seasonal_periods']
                    ).fit()
                elif model == "Holt":
                    model_instance = Holt(train, exponential=params['exponential']).fit()
                elif model == "LinearRegression":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = LinearRegression()
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "XGBRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = XGBRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        learning_rate=params['learning_rate'],
                        objective='reg:squarederror',
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "ARIMA":
                    order = (params['p'], params['d'], params['q'])
                    model_instance = ARIMA(train, order=order).fit()
                elif model == "RandomForestRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = RandomForestRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "LSTM":
                    model_instance = Sequential([
                        LSTM(params['units'], activation='relu', input_shape=(1, 1)),
                        Dense(1)
                    ])
                    model_instance.compile(optimizer='adam', loss='mae')
                    train_reshaped = train.values.reshape(-1, 1, 1)
                    model_instance.fit(train_reshaped, train, epochs=params['epochs'], batch_size=1, verbose=0)
                    test_reshaped = np.arange(len(train), len(train) + len(test)).reshape(-1, 1, 1)
                    forecast = model_instance.predict(test_reshaped).flatten()
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue

                forecast = model_instance.forecast(steps=len(test))
                error = mean_absolute_error(test, forecast)
                results.append(error)
            except Exception as e:
                results.append(100000000)  # Assign a high error value for exceptions
        return results

    # SARIMA
    try:
        model = "SARIMA"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        sarima_results = tuner.minimize()

        best_params = sarima_results['best_params']
        best_order = (best_params['p'], best_params['d'], best_params['q'])
        best_seasonal_order = (best_params['P'], best_params['D'], best_params['Q'], best_params['s'])
        sarima_model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order).fit(disp=False)
        sarima_forecast = sarima_model.forecast(steps=len(test))
        results['SARIMA'] = (mean_absolute_error(test, sarima_forecast), best_params)
    except Exception as e:
        results['SARIMA'] = (float('inf'), None)

    # Exponential Smoothing
    try:
        model = "ExponentialSmoothing"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        es_results = tuner.minimize()

        best_params = es_results['best_params']
        es_model = ExponentialSmoothing(
            train,
            seasonal=best_params['seasonal'],
            seasonal_periods=best_params['seasonal_periods']
        ).fit()
        es_forecast = es_model.forecast(steps=len(test))
        results['Exponential Smoothing'] = (mean_absolute_error(test, es_forecast), best_params)
    except Exception as e:
        results['Exponential Smoothing'] = (float('inf'), None)

    # Holt's Linear Trend
    try:
        model = "Holt"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        holt_results = tuner.minimize()

        best_params = holt_results['best_params']
        holt_model = Holt(train, exponential=best_params['exponential']).fit()
        holt_forecast = holt_model.forecast(steps=len(test))
        results["Holt's Linear Trend"] = (mean_absolute_error(test, holt_forecast), best_params)
    except Exception as e:
        results["Holt's Linear Trend"] = (float('inf'), None)

    # XGBoost
    try:
        model = "XGBRegressor"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        xgb_results = tuner.minimize()

        best_params = xgb_results['best_params']
        X_train = np.arange(len(train)).reshape(-1, 1)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        xgb_model = XGBRegressor(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            objective='reg:squarederror',
            random_state=42
        )
        xgb_model.fit(X_train, train)
        xgb_forecast = xgb_model.predict(X_test)
        results['XGBoost'] = (mean_absolute_error(test, xgb_forecast), best_params)
    except Exception as e:
        results['XGBoost'] = (float('inf'), None)

    # ARIMA
    try:
        model = "ARIMA"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        arima_results = tuner.minimize()

        best_params = arima_results['best_params']
        best_order = (best_params['p'], best_params['d'], best_params['q'])
        arima_model = ARIMA(train, order=best_order).fit()
        arima_forecast = arima_model.forecast(steps=len(test))
        results['ARIMA'] = (mean_absolute_error(test, arima_forecast), best_params)
    except Exception as e:
        results['ARIMA'] = (float('inf'), None)

    # Linear Regression
    try:
        model = "LinearRegression"
        param_space = {}  # No hyperparameters to tune for Linear Regression
        optimizer_config = {'initial_random': 1, 'num_iteration': 1}
        tuner = Tuner(param_space, objective_function, optimizer_config)
        lr_results = tuner.minimize()

        X_train = np.arange(len(train)).reshape(-1, 1)
        lr_model = LinearRegression()
        lr_model.fit(X_train, train)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        lr_forecast = lr_model.predict(X_test)
        results['Linear Regression'] = (mean_absolute_error(test, lr_forecast), None)
    except Exception as e:
        results['Linear Regression'] = (float('inf'), None)

    # Random Forest
    try:
        model = "RandomForestRegressor"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        rf_results = tuner.minimize()

        best_params = rf_results['best_params']
        X_train = np.arange(len(train)).reshape(-1, 1)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        rf_model = RandomForestRegressor(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            random_state=42
        )
        rf_model.fit(X_train, train)
        rf_forecast = rf_model.predict(X_test)
        results['RandomForestRegressor'] = (mean_absolute_error(test, rf_forecast), best_params)
    except Exception as e:
        results['RandomForestRegressor'] = (float('inf'), None)

    # LSTM
    try:
        model = "LSTM"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        lstm_results = tuner.minimize()

        best_params = lstm_results['best_params']
        lstm_model = Sequential([
            LSTM(best_params['units'], activation='relu', input_shape=(1, 1)),
            Dense(1)
        ])
        lstm_model.compile(optimizer='adam', loss='mae')
        train_reshaped = train.values.reshape(-1, 1, 1)
        lstm_model.fit(train_reshaped, train, epochs=best_params['epochs'], batch_size=1, verbose=0)
        test_reshaped = np.arange(len(train), len(train) + len(test)).reshape(-1, 1, 1)
        lstm_forecast = lstm_model.predict(test_reshaped).flatten()
        results['LSTM'] = (mean_absolute_error(test, lstm_forecast), best_params)
    except Exception as e:
        results['LSTM'] = (float('inf'), None)

    return results


In [32]:
# Inverse dictionary
def inverse_dict(d):
    return {v: k for k, v in d.items()}

In [33]:
@contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

In [34]:
# Find the best model for each cluster
clusters_centroid = inverse_dict(cluster[cluster["centroid"] == True]["cluster"].to_dict())
clusters_model ={}
import time

timer = time.time()
timing_row_starting = [time.strftime("%H:%M:%S", time.gmtime(time.time())), 'finding clusteroid optimized model', None, None]

for c, i in clusters_centroid.items():
    train = df_train[df_train["unique_id"] == i].y
    test = df_test[df_test["unique_id"] == i].y
    with suppress_output():
        mae = evaluate_models_on_centroid(train, test)
    best_model = min(mae.items(), key=lambda x: x[1][0])
    clusters_model[c] = (best_model[0], mae[best_model[0]][1])

    print(f"Cluster {c}: Best Model = {best_model[0]} (MAE = {best_model[1][0]:.2f})")

timing_row_ending = [None, 'finding clusteroid optimized model', time.strftime("%H:%M:%S", time.gmtime(time.time())), time.time() - timer]

timing = pd.concat([timing, pd.DataFrame([timing_row_starting, timing_row_ending], columns=timing.columns)])

timing.to_csv("timing.csv")

Cluster 0: Best Model = RandomForestRegressor (MAE = 0.00)
Cluster 2: Best Model = ARIMA (MAE = 22.71)
Cluster 1: Best Model = SARIMA (MAE = 7.26)


In [35]:
clusters_model_df = pd.DataFrame(
	[(cluster, model, params) for cluster, (model, params) in clusters_model.items()],
	columns=["cluster", "model", "params"]
)

In [36]:
clusters_model_df.to_csv("data/clusters_model.csv")